# Data processing test

#### Maria Silva, Feb. 2025

This notebook contains an example showing our data processing pipeline, using a single day of data.

In [1]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Load data

### Mempool dumpster

This data can be downloaded freely from [mempool-dumpster.flashbots.net]
(https://mempool-dumpster.flashbots.net/index.html).

**Relevant columns**

- `timestamp_ms`: timestamp when the transaction was first seen by a source (in milliseconds)
- `value`: transaction value (in wei)
- `gas`: gas limit of the transaction (in units of gas)
- `gas_price`: not sure... seems to be the same as `gas_fee_cap` as it is always equal
- `gas_tip_cap`: same as max. priority fee, which is the max. the user is willing to give to the block producer (in wei)
- `gas_fee_cap`: max. gas price the user is willing to pay (in wei)
- `data_size`: size of the input data filed for the transaction
- `sources`: mempool sources where the transaction apeared
- `included_at_block_height`: height of the block where the transaction was included
- `included_block_timestamp_ms`: timestamp of the block where the transaction was included (in milliseconds)
- `inclusion_delay_ms`: `included_block_timestamp_ms` - `timestamp_ms` (in milliseconds)

**Relevant conversions**

- 1 wei = 1e-9 gwei
- 1 wei = 1e-18 eth

Now, let's load one day of this data to test it (2025-01-01).

In [2]:
file_name = os.path.abspath("../data/2025-01-01.csv")

tx_df = pd.read_csv(file_name)

tx_df = tx_df[tx_df["sources"].str.contains("local")]

tx_df["arrival_time"] = pd.to_datetime(tx_df["timestamp_ms"], unit="ms")
tx_df["included_block_timestamp_ms"] = pd.to_datetime(tx_df["included_block_timestamp_ms"], unit="ms")

select_cols = ["arrival_time", "hash", "gas", "gas_tip_cap", "gas_fee_cap", "data_size",
               "inclusion_delay_ms", "included_at_block_height"]
tx_df = tx_df[select_cols].sort_values("arrival_time")

tx_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 815743 entries, 0 to 845947
Data columns (total 8 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   arrival_time              815743 non-null  datetime64[ns]
 1   hash                      815743 non-null  object        
 2   gas                       815743 non-null  int64         
 3   gas_tip_cap               815743 non-null  int64         
 4   gas_fee_cap               815743 non-null  int64         
 5   data_size                 815743 non-null  int64         
 6   inclusion_delay_ms        815743 non-null  int64         
 7   included_at_block_height  815743 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 56.0+ MB


### Block data

We extracted 6 months of block data from [Google Big Query](https://console.cloud.google.com/bigquery?inv=1&invt=Abp-yA&project=ethereum-public-data-407811&ws=!1m0). The query can be found in the `data` folder in `bigquery_eth_blocks_gas.sql`.

In [3]:
blocks_df = pd.read_csv("../data/eth_blocks_gas_6_months.csv")
blocks_df["timestamp"] = pd.to_datetime(blocks_df["timestamp"]).dt.tz_localize(None)
blocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322689 entries, 0 to 1322688
Data columns (total 7 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   timestamp          1322689 non-null  datetime64[ns]
 1   block_number       1322689 non-null  int64         
 2   size_bytes         1322689 non-null  int64         
 3   gas_used           1322689 non-null  int64         
 4   blob_gas_used      1322689 non-null  int64         
 5   transaction_count  1322689 non-null  int64         
 6   base_fee_gwei      1322689 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(5)
memory usage: 70.6 MB


## Combine block and transaction data

In [4]:
# compute previous block height and add to txs data
block_times = blocks_df[["timestamp", "block_number"]]
block_times.columns = ["prev_block_time", "prev_block_height"]
tx_bl_df = pd.merge_asof(
    tx_df, block_times, left_on="arrival_time", right_on="prev_block_time", direction="backward"
    )
tx_bl_df["inclusion_delay_blocks"] = tx_bl_df["included_at_block_height"]-tx_bl_df["prev_block_height"]
tx_bl_df = tx_bl_df[tx_bl_df["inclusion_delay_blocks"]>0].reset_index(drop=True)

# add info on last block since tx arrival
tx_bl_df = tx_bl_df.merge(blocks_df, how="left", left_on="prev_block_height", right_on="block_number")
tx_bl_df = tx_bl_df.drop(columns=["timestamp", "block_number"])
tx_bl_df = tx_bl_df.rename(columns={
    "size_bytes": "prev_block_size_bytes",
    "gas_used": "prev_block_gas_used",
    "transaction_count": "prev_block_tx_count",
    "base_fee_gwei": "prev_block_base_fee_gwei",

})
tx_bl_df.head()

,arrival_time,hash,gas,gas_tip_cap,gas_fee_cap,data_size,inclusion_delay_ms,included_at_block_height,prev_block_time,prev_block_height,inclusion_delay_blocks,prev_block_size_bytes,prev_block_gas_used,blob_gas_used,prev_block_tx_count,prev_block_base_fee_gwei
0,2025-01-01 00:00:00.228,0x01f2bad1d1051948affae31e92232cc2468b11cf76a0...,178080,3000000000,8221644068,708,10772,21525891,2024-12-31 23:59:59,21525890,1,75395,16274933,0,148,41.257435
1,2025-01-01 00:00:00.450,0x41f3168108b35ecad8338b87ebcb4efa300958ca1f99...,21000,16729008,4125844901,0,94550,21525898,2024-12-31 23:59:59,21525890,8,75395,16274933,0,148,41.257435
2,2025-01-01 00:00:00.452,0x1d92fa93d8ddf5cefa0f32a9d138162ff052cdfd2835...,30000,57000000,8310000000,0,10548,21525891,2024-12-31 23:59:59,21525890,1,75395,16274933,0,148,41.257435
3,2025-01-01 00:00:00.468,0xd9abbd7170fc25fe03e26f939fb8b82ab2f85cb2f57e...,59817,1000000000,41263615010,68,10532,21525891,2024-12-31 23:59:59,21525890,1,75395,16274933,0,148,41.257435
4,2025-01-01 00:00:00.538,0x7a8da8a254957575c7e827475badb4f64775d297633c...,50000,2001000000,999000000000,0,10462,21525891,2024-12-31 23:59:59,21525890,1,75395,16274933,0,148,41.257435


## Add mempool data

In [5]:
cols = ["hash", "gas", "gas_tip_cap", "gas_fee_cap", "data_size", "prev_block_height",
         "inclusion_delay_blocks", "included_at_block_height"]
mem_df = tx_bl_df[cols]
mem_df = mem_df.iloc[mem_df.index.repeat(mem_df['inclusion_delay_blocks'])].reset_index(drop=True)
mem_df["in_mempool_at_height"] = np.concat(
    [np.arange(start, end) for start, end in zip(
        tx_bl_df["prev_block_height"], tx_bl_df["included_at_block_height"]
        )]
    )
mem_df.head()

,hash,gas,gas_tip_cap,gas_fee_cap,data_size,prev_block_height,inclusion_delay_blocks,included_at_block_height,in_mempool_at_height
0,0x01f2bad1d1051948affae31e92232cc2468b11cf76a0...,178080,3000000000,8221644068,708,21525890,1,21525891,21525890
1,0x41f3168108b35ecad8338b87ebcb4efa300958ca1f99...,21000,16729008,4125844901,0,21525890,8,21525898,21525890
2,0x41f3168108b35ecad8338b87ebcb4efa300958ca1f99...,21000,16729008,4125844901,0,21525890,8,21525898,21525891
3,0x41f3168108b35ecad8338b87ebcb4efa300958ca1f99...,21000,16729008,4125844901,0,21525890,8,21525898,21525892
4,0x41f3168108b35ecad8338b87ebcb4efa300958ca1f99...,21000,16729008,4125844901,0,21525890,8,21525898,21525893


In [6]:
mem_agg_df = mem_df.groupby("in_mempool_at_height")[["gas", "data_size"]].sum()
mem_agg_df["tx_count"] = mem_df.groupby("in_mempool_at_height").size()
gas_fee_stats_df = (
    mem_df
    .groupby("in_mempool_at_height")
    ["gas_fee_cap"]
    .quantile(np.arange(0, 1.2, 0.2).round(1))
    .unstack()
    )
gas_tip_stats_df = (
    mem_df
    .groupby("in_mempool_at_height")
    ["gas_tip_cap"]
    .quantile(np.arange(0, 1.2, 0.2).round(1))
    .unstack()
    )
mem_agg_df = pd.concat([mem_agg_df, gas_fee_stats_df, gas_tip_stats_df], axis=1).reset_index()
mem_agg_df.columns = (
    ["block_height", "mem_total_gas_limit", "mem_total_data_size", "mem_tx_count"] +
    [f"mem_gas_fee_cap_{c}" for c in gas_fee_stats_df.columns] +
    [f"mem_gas_tip_cap_{c}" for c in gas_tip_stats_df.columns]
    )
mem_agg_df.head()

,block_height,mem_total_gas_limit,mem_total_data_size,mem_tx_count,mem_gas_fee_cap_0.0,mem_gas_fee_cap_0.2,mem_gas_fee_cap_0.4,mem_gas_fee_cap_0.6,mem_gas_fee_cap_0.8,mem_gas_fee_cap_1.0,mem_gas_tip_cap_0.0,mem_gas_tip_cap_0.2,mem_gas_tip_cap_0.4,mem_gas_tip_cap_0.6,mem_gas_tip_cap_0.8,mem_gas_tip_cap_1.0
0,21525890,45970613,56174,213,4.125743e+09,5.003829e+09,6.937063e+09,2.400000e+10,2.400000e+10,1.000000e+12,863637.0,817978507.4,1.400000e+09,1.400000e+09,2.000000e+09,1.219250e+10
1,21525891,30199284,55777,140,3.395659e+09,4.947175e+09,5.682682e+09,8.250000e+09,1.213021e+10,5.020000e+11,863637.0,25333000.0,8.079828e+08,2.000000e+09,4.130810e+09,1.219250e+10
2,21525892,20070857,21409,121,3.395659e+09,4.337351e+09,5.145495e+09,6.873534e+09,8.820000e+09,1.820000e+11,6079920.0,100000000.0,1.000000e+09,2.000000e+09,4.180000e+09,1.320000e+10
3,21525893,25759195,27691,128,3.395659e+09,4.312078e+09,5.117889e+09,6.020457e+09,8.006886e+09,1.000000e+12,863637.0,13910125.8,8.066207e+08,2.000000e+09,4.174644e+09,1.219250e+10
4,21525894,26997126,26451,185,3.395659e+09,4.534141e+09,5.160719e+09,8.248494e+09,6.560652e+10,1.000000e+12,863637.0,9500000.0,9.880000e+08,2.000000e+09,4.174644e+09,1.219250e+10


In [7]:
tx_bl_mem_df = (
    tx_bl_df
    .merge(mem_agg_df, how="left", left_on="prev_block_height", right_on="block_height")
    .drop(columns="block_height")
    )
tx_bl_mem_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 626967 entries, 0 to 626966
Data columns (total 31 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   arrival_time              626967 non-null  datetime64[ns]
 1   hash                      626967 non-null  object        
 2   gas                       626967 non-null  int64         
 3   gas_tip_cap               626967 non-null  int64         
 4   gas_fee_cap               626967 non-null  int64         
 5   data_size                 626967 non-null  int64         
 6   inclusion_delay_ms        626967 non-null  int64         
 7   included_at_block_height  626967 non-null  int64         
 8   prev_block_time           626967 non-null  datetime64[ns]
 9   prev_block_height         626967 non-null  int64         
 10  inclusion_delay_blocks    626967 non-null  int64         
 11  prev_block_size_bytes     626967 non-null  int64         
 12  pr